## Google Playstore

In [1]:
from google_play_scraper import app, Sort, reviews_all
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json, os, uuid

In [3]:
g_reviews = reviews_all(
        "com.bobobox.bobobox",
        sleep_milliseconds=0, # defaults to 0
        lang='en', # defaults to 'en'
        country='id', # defaults to 'us'
        sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
    )

g_df = pd.DataFrame(np.array(g_reviews),columns=['review'])
g_df2 = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))

g_df2.drop(columns={'userImage', 'reviewCreatedVersion'},inplace = True)
g_df2.rename(columns= {'score': 'rating','userName': 'user_name', 'reviewId': 'review_id', 'content': 'review_description', 'at': 'review_date', 'replyContent': 'developer_response', 'repliedAt': 'developer_response_date', 'thumbsUpCount': 'thumbs_up'},inplace = True)
g_df2.insert(loc=0, column='source', value='Google Play')
g_df2.insert(loc=3, column='review_title', value=None)
g_df2['language_code'] = 'en'
g_df2['country_code'] = 'id'

In [9]:
g_df2.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code
0,Google Play,f489e856-5d12-48a8-b36e-80d7c6d403ab,Yayu Nur afianti,None,nyaman,5,0,2023-12-02 15:10:04,None,NaT,4.6.4,en,id
1,Google Play,d08ca1e1-9365-44f3-a92f-e5171015bcec,Aisyah Idrus,None,Best aplikasi untuk penginapan yg terupdate da...,5,0,2023-12-01 08:48:57,None,NaT,4.6.5,en,id
2,Google Play,2afe99a1-e743-4075-94bf-d510468abb13,Lodwyik Nico,None,"i got error, when click use voucher failed i c...",5,0,2023-11-30 19:18:34,None,NaT,4.6.5,en,id
3,Google Play,4b2551f6-a692-4af3-b597-b18dd5dab5f2,Dwi Budi Winarno,None,loyal friend when traveling solo,5,0,2023-11-27 09:49:01,None,NaT,4.6.2,en,id
4,Google Play,a7af8a72-fa4a-4379-8e79-5dfd323caf24,Athariq Zhafari Aufar,None,absolute beauty.,5,0,2023-11-13 14:23:18,None,NaT,None,en,id


## App Store

In [5]:
from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json, os, uuid

a_reviews = AppStore('id', 'bobobox', '1401179501')
a_reviews.review()

a_df = pd.DataFrame(np.array(a_reviews.reviews),columns=['review'])
a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))

a_df2.drop(columns={'isEdited'},inplace = True)
a_df2.insert(loc=0, column='source', value='App Store')
a_df2['developer_response_date'] = None
a_df2['thumbs_up'] = None
a_df2['language_code'] = 'en'
a_df2['country_code'] = 'id'
a_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(a_df2.index))])
a_df2.rename(columns= {'review': 'review_description','userName': 'user_name', 'date': 'review_date','title': 'review_title', 'developerResponse': 'developer_response'},inplace = True)
a_df2 = a_df2.where(pd.notnull(a_df2), None)

a_df2

2023-12-08 17:25:26,370 [INFO] Base - Initialised: AppStore('id', 'bobobox', 1401179501)
2023-12-08 17:25:26,372 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/id/app/bobobox/id1401179501
2023-12-08 17:25:29,361 [INFO] Base - [id:1401179501] Fetched 177 reviews (177 fetched in total)


,source,review_id,review_date,developer_response,review_description,rating,review_title,user_name,developer_response_date,thumbs_up,language_code,country_code
0,App Store,e0caeb51-8c11-41de-bd49-8a387608121f,2019-09-06 08:14:01,"{'id': 18138351, 'body': 'Thank you for the fe...","Nyaman dan seru bgt nginap di bobobox, dngn ha...",5,Luv,Fitriyani hidayanti,None,None,en,id
1,App Store,26b230f7-1b12-486a-8a11-374f079b970b,2018-09-23 05:24:26,"{'id': 10223415, 'body': 'Thank you for the fe...","Easy to use, amazing apps !",5,Awesome apps,Julie2289,None,None,en,id
2,App Store,8fa967c7-bda4-421e-845d-b9bc2721c73f,2018-08-07 10:00:56,"{'id': 10223646, 'body': 'Thank you for the fe...","semakin keren nih Bobobox, akhirnya udah ada d...",5,Juara !!!,dikasptra,None,None,en,id
3,App Store,744ddeb4-d0c7-4a49-821b-f07d5db8837b,2020-01-07 05:20:29,"{'id': 17507192, 'body': 'Thank you for the fe...","love it, aplikasinya semakin membaik dan semak...",5,Love it,LionyDionita,None,None,en,id
4,App Store,7eea3017-42e5-401e-a769-bc43e19e47c2,2020-08-24 06:19:26,"{'id': 17507140, 'body': 'Thank you for the fe...",Really affordable & have a good strategic loca...,5,Love it !,irahadian,None,None,en,id
...,...,...,...,...,...,...,...,...,...,...,...,...
172,App Store,c20e7e25-212c-445b-a1c8-ffc629361279,2020-01-31 06:55:46,"{'id': 17507182, 'body': 'Hai Kak, Mohon maaf ...",How ro change ID card photo?\nWhile upload ID ...,1,ID Card photo,iketutwiyasa,None,None,en,id
173,App Store,f3398ebe-d9c1-47fb-b903-275e0a6f2304,2020-06-11 13:22:03,"{'id': 17132189, 'body': 'Hello! Thank you ...",Worst!,1,Bad customer services!,Jajajajajahahjaja,None,None,en,id
174,App Store,4fe11cd8-1a86-4c94-8910-ad869ed03497,2020-06-11 03:59:43,"{'id': 17132171, 'body': 'Hai Kak Seleketeb, M...",Tidak ada bahasa indonesianya,1,Y,seleketeb,None,None,en,id
175,App Store,3abc05b8-1a96-4722-a517-43e106885234,2020-05-07 12:04:10,"{'id': 17507171, 'body': 'Hai Kak Banana, Moho...",Aplikasi tidak bisa dibuka di iphone 5!,1,App tidak bisa dibuka,Mr Banana 5987,None,None,en,id


## Concat Google Plasystore & App Store

In [8]:
result = pd.concat([g_df2,a_df2])
result

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,appVersion,language_code,country_code
0,Google Play,f489e856-5d12-48a8-b36e-80d7c6d403ab,Yayu Nur afianti,None,nyaman,5,0,2023-12-02 15:10:04,None,NaT,4.6.4,en,id
1,Google Play,d08ca1e1-9365-44f3-a92f-e5171015bcec,Aisyah Idrus,None,Best aplikasi untuk penginapan yg terupdate da...,5,0,2023-12-01 08:48:57,None,NaT,4.6.5,en,id
2,Google Play,2afe99a1-e743-4075-94bf-d510468abb13,Lodwyik Nico,None,"i got error, when click use voucher failed i c...",5,0,2023-11-30 19:18:34,None,NaT,4.6.5,en,id
3,Google Play,4b2551f6-a692-4af3-b597-b18dd5dab5f2,Dwi Budi Winarno,None,loyal friend when traveling solo,5,0,2023-11-27 09:49:01,None,NaT,4.6.2,en,id
4,Google Play,a7af8a72-fa4a-4379-8e79-5dfd323caf24,Athariq Zhafari Aufar,None,absolute beauty.,5,0,2023-11-13 14:23:18,None,NaT,None,en,id
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,App Store,c20e7e25-212c-445b-a1c8-ffc629361279,iketutwiyasa,ID Card photo,How ro change ID card photo?\nWhile upload ID ...,1,None,2020-01-31 06:55:46,"{'id': 17507182, 'body': 'Hai Kak, Mohon maaf ...",None,NaN,en,id
173,App Store,f3398ebe-d9c1-47fb-b903-275e0a6f2304,Jajajajajahahjaja,Bad customer services!,Worst!,1,None,2020-06-11 13:22:03,"{'id': 17132189, 'body': 'Hello! Thank you ...",None,NaN,en,id
174,App Store,4fe11cd8-1a86-4c94-8910-ad869ed03497,seleketeb,Y,Tidak ada bahasa indonesianya,1,None,2020-06-11 03:59:43,"{'id': 17132171, 'body': 'Hai Kak Seleketeb, M...",None,NaN,en,id
175,App Store,3abc05b8-1a96-4722-a517-43e106885234,Mr Banana 5987,App tidak bisa dibuka,Aplikasi tidak bisa dibuka di iphone 5!,1,None,2020-05-07 12:04:10,"{'id': 17507171, 'body': 'Hai Kak Banana, Moho...",None,NaN,en,id


In [47]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 553 entries, 0 to 176
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   source                 553 non-null    object        
 1   review_id              553 non-null    object        
 2   user_name              553 non-null    object        
 3   review_title           177 non-null    object        
 4   review_description     553 non-null    object        
 5   rating                 553 non-null    int64         
 6   thumbs_up              376 non-null    object        
 7   review_date            553 non-null    datetime64[ns]
 8   review_sentiment       553 non-null    object        
 9   review_description_en  553 non-null    object        
 10  rating_sentiment       553 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 68.0+ KB


In [46]:
result['rating_sentiment'] = result['rating'].apply(lambda x: 'Positif' if x > 4 else ('Netral' if x == 3 else 'Negatif'))
result[['rating_sentiment', 'rating']]

,rating_sentiment,rating
0,Positif,5
1,Positif,5
2,Positif,5
3,Positif,5
4,Positif,5
...,...,...
172,Negatif,1
173,Negatif,1
174,Negatif,1
175,Negatif,1


In [51]:
result.drop(['developer_response', 'developer_response_date', 'appVersion', 'language_code', 'country_code'], inplace=True, axis=1)
result.head()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,review_description_en,rating_sentiment
0,Google Play,f489e856-5d12-48a8-b36e-80d7c6d403ab,Yayu Nur afianti,None,nyaman,5,0,2023-12-02 15:10:04,nyaman,Positif
1,Google Play,d08ca1e1-9365-44f3-a92f-e5171015bcec,Aisyah Idrus,None,Best aplikasi untuk penginapan yg terupdate da...,5,0,2023-12-01 08:48:57,Best aplikasi untuk penginapan yg terupdate da...,Positif
2,Google Play,2afe99a1-e743-4075-94bf-d510468abb13,Lodwyik Nico,None,"i got error, when click use voucher failed i c...",5,0,2023-11-30 19:18:34,"i got error, when click use voucher failed i c...",Positif
3,Google Play,4b2551f6-a692-4af3-b597-b18dd5dab5f2,Dwi Budi Winarno,None,loyal friend when traveling solo,5,0,2023-11-27 09:49:01,loyal friend when traveling solo,Positif
4,Google Play,a7af8a72-fa4a-4379-8e79-5dfd323caf24,Athariq Zhafari Aufar,None,absolute beauty.,5,0,2023-11-13 14:23:18,absolute beauty.,Positif


In [52]:
result.to_csv('scraping_data_store.csv', index=False)

In [56]:
df_scrape = pd.read_csv('D:\GitHub\TheOX47\Sentiment-Analysis\scraping_data_store.csv')
df_scrape.head()

,source,review_id,user_name,review_title,review_description,review_description_en,rating,thumbs_up,review_date,rating_sentiment
0,Google Play,f489e856-5d12-48a8-b36e-80d7c6d403ab,Yayu Nur afianti,NaN,nyaman,comfortable,5,0.0,2023-12-02 15:10:04,Positif
1,Google Play,d08ca1e1-9365-44f3-a92f-e5171015bcec,Aisyah Idrus,NaN,Best aplikasi untuk penginapan yg terupdate da...,Best Applications for updated and shocked lodg...,5,0.0,2023-12-01 8:48:57,Positif
2,Google Play,2afe99a1-e743-4075-94bf-d510468abb13,Lodwyik Nico,NaN,"i got error, when click use voucher failed i c...","I Got Error, When Click Use Voucher Failed I C...",5,0.0,2023-11-30 19:18:34,Positif
3,Google Play,4b2551f6-a692-4af3-b597-b18dd5dab5f2,Dwi Budi Winarno,NaN,loyal friend when traveling solo,Loyal friend when traveling solo,5,0.0,2023-11-27 9:49:01,Positif
4,Google Play,a7af8a72-fa4a-4379-8e79-5dfd323caf24,Athariq Zhafari Aufar,NaN,absolute beauty.,Absolute Beauty.,5,0.0,2023-11-13 14:23:18,Positif


## Compound Score using Model VADER

In [57]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

review_col = df_scrape['review_description_en'].tolist()
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis and create DataFrame
sentiments = []
compound_scores = []

for sentence in review_col:
    sentiment_scores = sia.polarity_scores(sentence)
    compound_score = sentiment_scores['compound']
    
    if compound_score >= 0.05:
        sentiment = 'Positif'
    elif compound_score <= -0.05:
        sentiment = 'Negatif'
    else:
        sentiment = 'Netral'
    
    sentiments.append(sentiment)
    compound_scores.append(compound_score)


df_scrape['sentiment_vader'] = sentiments
df_scrape['compound_score'] = compound_scores

df_scrape.head()

,source,review_id,user_name,review_title,review_description,review_description_en,rating,thumbs_up,review_date,rating_sentiment,sentiment_vader,compound_score
0,Google Play,f489e856-5d12-48a8-b36e-80d7c6d403ab,Yayu Nur afianti,NaN,nyaman,comfortable,5,0.0,2023-12-02 15:10:04,Positif,Positif,0.5106
1,Google Play,d08ca1e1-9365-44f3-a92f-e5171015bcec,Aisyah Idrus,NaN,Best aplikasi untuk penginapan yg terupdate da...,Best Applications for updated and shocked lodg...,5,0.0,2023-12-01 8:48:57,Positif,Positif,0.6597
2,Google Play,2afe99a1-e743-4075-94bf-d510468abb13,Lodwyik Nico,NaN,"i got error, when click use voucher failed i c...","I Got Error, When Click Use Voucher Failed I C...",5,0.0,2023-11-30 19:18:34,Positif,Negatif,-0.7184
3,Google Play,4b2551f6-a692-4af3-b597-b18dd5dab5f2,Dwi Budi Winarno,NaN,loyal friend when traveling solo,Loyal friend when traveling solo,5,0.0,2023-11-27 9:49:01,Positif,Positif,0.7430
4,Google Play,a7af8a72-fa4a-4379-8e79-5dfd323caf24,Athariq Zhafari Aufar,NaN,absolute beauty.,Absolute Beauty.,5,0.0,2023-11-13 14:23:18,Positif,Positif,0.5859


In [62]:
df_scrape.to_csv('scraping_data_store.csv', index=False)